# **Genetic Algorithm - Phrase Solver**

This notebook shows how to use the genetic algorithm classes to create a training scenario.
It will demonstrate how a genetic algorithm can be used to generate a specific sentence.

For a sentence of length $L$ constructed from $N_C$ possible characters, the number of permutations of possible sentences is given by $(N_C)^L$.

If $N_P$ sentences are created each generation, it would take $\frac{(N_C)^L}{N_P}$ generations on average to create a specific sentence.
This number grows large quickly, and so this genetic algorithm aims to implement a more efficient algorithm to find a solution.

## Code Implementation

### Libraries and Helper Functions

To create the training scenario, we will need to import `GeneticAlgorithm` and `Member`. We will use class inheritance to set things up.

In [1]:
from __future__ import annotations
from typing import List
import numpy as np
import datetime

from src.ga import GeneticAlgorithm
from src.member import Member

We can use the following helper function to analyse the results from the training algorithm.

In [2]:
def print_system_msg(msg: str) -> None:
    """
    Print a message to the terminal.

    Parameters:
        msg (str): Message to print
    """
    print(f"[{datetime.datetime.now().strftime('%d-%m-%G | %H:%M:%S')}] {msg}")

### Creating Classes from Member and GeneticAlgorithm

We will need to configure the chromosome, fitness function and crossover for the members.

In this case, the chromosome is the generated sentence.
The crossover function randomly selects from 2 members' sentences with a chance to select a random character.
The fitness function measures how many characters in the generated sentence match a specified phrase.

The crossover function must generate `_new_chromosome`.

In [3]:
class PhraseSolverMember(Member):
    """
    Member to use in PhraseSolver app.
    """

    def __init__(self, length: int, gene_pool: List[str]) -> None:
        """
        Initialise PhraseSolverMember with length of phrase and gene pool.

        Parameters:
            length (int): Length of member chromosome
            gene_pool (List[str]): List of possible characters
        """
        super().__init__()
        self._length = length
        self._gene_pool = gene_pool
        self._chromosome = "".join([self.random_char for _ in range(self._length)])

    @property
    def random_char(self) -> str:
        """
        Return a random gene from the possible genes.
        """
        _choice: str = np.random.choice(self._gene_pool)
        return _choice

    @property
    def fitness(self) -> int:
        """
        Return member fitness.
        """
        return self._score**2

    def calculate_score(self, phrase: str) -> None:
        """
        Calculate the member's score based on the provided phrase.

        Parameters:
            phrase (str): Used to compare chromosome to phrase and calculate fitness
        """
        self._score = sum([self._chromosome[i] == phrase[i] for i in range(self._length)])

    def crossover(self, parent_a: Member, parent_b: Member, mutation_rate: int) -> None:
        """
        Crossover the chromosomes of two parents to create a new chromosome.

        Parameters:
            parent_a (Member): Used to construct new chromosome
            parent_b (Member): Used to construct new chromosome
            mutation_rate (int): Probability for mutations to occur
        """
        self._new_chromosome = ""

        for i in range(self._length):
            prob = np.random.randint(0, 100)

            # Half of the genes will come from parentA
            if prob < (100 - mutation_rate) / 2:
                new_char = parent_a._chromosome[i]
            # Half of the genes will come from parentB
            elif prob < (100 - mutation_rate):
                new_char = parent_b._chromosome[i]
            # Chance for a random genes to be selected
            else:
                new_char = self.random_char

            self._new_chromosome += new_char

Next, we need to set up the training algorithm.
We can create a List of Members to use for the population.
We then need to define how to evaluate the fitness of each member in this population, and how to analyse the generation.
In this case, we will evaluate the population against a specified phrase and print the best chromosome from each generation.

In [4]:
class PhraseSolver(GeneticAlgorithm):
    """
    Simple app to use genetic algorithms to solve an alphanumeric phrase.
    """

    def __init__(self, mutation_rate: int) -> None:
        """
        Initialise PhraseSolver app.

        Parameters:
            mutation_rate (int)
        """
        super().__init__(mutation_rate)
        self._phrase: str

    @classmethod
    def create_and_run(
        cls, population_size: int, mutation_rate: int, phrase: str, mem_genes: List[str]
    ) -> PhraseSolver:
        """
        Create app and run genetic algorithm.

        Parameters:
            population_size (int): Number of members in population
            mutation_rate (int): Mutation rate for members
            phrase (str): Phrase for members to solve
            mem_genes (List[str]): List of possible member genes

        Returns:
            ga (PhraseSolver): Phrase solver app
        """
        ga = cls(mutation_rate)
        ga._phrase = phrase
        ga._add_population([PhraseSolverMember(len(ga._phrase), mem_genes) for _ in range(population_size)])
        print(ga)
        ga.run()
        return ga

    def _evaluate(self) -> None:
        """
        Evaluate the population.
        """
        self._population.calculate_member_scores(self._phrase)
        self._population.evaluate()

    def _analyse(self) -> None:
        """
        Analyse best member's chromosome.
        """
        _gen_text = f"Generation {self._generation:>4}:"

        # Correct phrase found so break out of the loop
        if self._population.best_chromosome == self._phrase:
            print_system_msg(f"{_gen_text} {self._population.best_chromosome} \t|| Solved!")
            self._running = False
            return

        # Return the closest match and its associated fitness then evolve.
        print_system_msg(
            f"{_gen_text} {self._population.best_chromosome} \t|| Max Fitness: {self._population.best_fitness}"
        )

### Running the Algorithm

Now we can specify a population size, mutation rate, phrase and genes for each member to select from.

In [5]:
population_size = 200
mutation_rate = 3
phrase = "I am a genetic algorithm!"
gene_pool = list("0123456789 abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,!?")

Each generation, the best guess will be printed and until eventually it has converged on the specified phrase.

In [14]:
ga = PhraseSolver.create_and_run(population_size, mutation_rate, phrase, gene_pool)

Population Size: 200
Mutation Rate: 3
[15-04-2024 | 19:55:06] Generation    1: mWjxRX Ce.eFeUhi xlny.Dlr 	|| Max Fitness: 9
[15-04-2024 | 19:55:06] Generation    2: f9lmOrQ8Bm6PU9 aNgcZimRa7 	|| Max Fitness: 25
[15-04-2024 | 19:55:06] Generation    3: I9lmOawlBm6l,9baNgctimRa7 	|| Max Fitness: 36
[15-04-2024 | 19:55:06] Generation    4: Iqam a CO9il.9qiP4lniQEWA 	|| Max Fitness: 49
[15-04-2024 | 19:55:06] Generation    5: mflmRaWq7nItiTUa1gNti6R0U 	|| Max Fitness: 64
[15-04-2024 | 19:55:06] Generation    6: IKam7a C99iqrX aOg Zim?m8 	|| Max Fitness: 100
[15-04-2024 | 19:55:06] Generation    7: Amag a geniP1Ubah7oaiQR3U 	|| Max Fitness: 100
[15-04-2024 | 19:55:06] Generation    8: m9am a geo,te9biNg7nimDWK 	|| Max Fitness: 100
[15-04-2024 | 19:55:06] Generation    9: IWym a 8en2tT7ka1gUZivWH! 	|| Max Fitness: 144
[15-04-2024 | 19:55:06] Generation   10: I ym a genetP7bing niFru5 	|| Max Fitness: 169
[15-04-2024 | 19:55:06] Generation   11: IJH3 a gemetFW sNg r8BRmO 	|| Max Fitness: 144
